---
title: "Polars as dplyr Introdution"
format:
  html:
    embed-resources: true
    mainfont: "Roboto"
    code-overflow: scroll
    code-line-numbers: true
highlight-style: darkly
css: styles.css
date: '2026-02-13'
---

## Load polars 🐻‍❄️
- `uv pip install polars`
- `uv pip install great_tables`
- In this notebook a will try to attempt to use most used `dplyr` verbs that I have widely used in `r` Tidyverse Ecosystem.

In [1]:
import polars as pl

## dataset

In [2]:
#| label: create dataset

df1 = pl.DataFrame(
    {
        "id": [1, 2, 3, 4, 5, 6],
        "group": ["A", "A", "B", "B", "C", "C"],
        "age": [25, 30, 22, 40, 35, 28],
        "income": [50000, 60000, 45000, 80000, 72000, 52000],
        "status": ["active", "inactive", "active", "active", "inactive", "active"],
        "prefix_score1": [10, 20, 30, 40, 50, 60],
        "prefix_score2": [5, 15, 25, 35, 45, 55],
    }
)

df2 = pl.DataFrame({"id": [1, 2, 3, 7], "city": ["NY", "LA", "Dallas", "Houston"]})

**column names as a the dataframe.**

In [3]:
pl.DataFrame({"Name of the dataframe": list(df1.columns)})

Name of the dataframe
str
"""id"""
"""group"""
"""age"""
"""income"""
"""status"""
"""prefix_score1"""
"""prefix_score2"""


## `.select()`

In [4]:
df1.select(["id", "age", "income"]).head(10)

id,age,income
i64,i64,i64
1,25,50000
2,30,60000
3,22,45000
4,40,80000
5,35,72000
6,28,52000


## `.filter()` 

In [5]:
df1.filter(pl.col("age") > 30)

id,group,age,income,status,prefix_score1,prefix_score2
i64,str,i64,i64,str,i64,i64
4,"""B""",40,80000,"""active""",40,35
5,"""C""",35,72000,"""inactive""",50,45


### `.filter()` with *`chain`* method

In [6]:
(
    df1
    .filter(pl.col("age")> 30)
    .select(["id", "group", "age", "income"])
)

id,group,age,income
i64,str,i64,i64
4,"""B""",40,80000
5,"""C""",35,72000


## `.with_columns()`

In [7]:
df1.with_columns(
    (pl.col("income")/1000).alias("income_k")
).select(['income', "income_k"])

income,income_k
i64,f64
50000,50.0
60000,60.0
45000,45.0
80000,80.0
72000,72.0
52000,52.0


__mutate with multiple columns__

In [8]:
df1.with_columns(
    pl.all().exclude(['status', 'group','id'])*2
)

id,group,age,income,status,prefix_score1,prefix_score2
i64,str,i64,i64,str,i64,i64
1,"""A""",50,100000,"""active""",20,10
2,"""A""",60,120000,"""inactive""",40,30
3,"""B""",44,90000,"""active""",60,50
4,"""B""",80,160000,"""active""",80,70
5,"""C""",70,144000,"""inactive""",100,90
6,"""C""",56,104000,"""active""",120,110


In [9]:
df1.with_columns(
    [
        (pl.col(col) * 2).alias(f"{col}_prod2")
        for col in df1.columns
        if col not in ["status", "group", "id"]
    ]
)

id,group,age,income,status,prefix_score1,prefix_score2,age_prod2,income_prod2,prefix_score1_prod2,prefix_score2_prod2
i64,str,i64,i64,str,i64,i64,i64,i64,i64,i64
1,"""A""",25,50000,"""active""",10,5,50,100000,20,10
2,"""A""",30,60000,"""inactive""",20,15,60,120000,40,30
3,"""B""",22,45000,"""active""",30,25,44,90000,60,50
4,"""B""",40,80000,"""active""",40,35,80,160000,80,70
5,"""C""",35,72000,"""inactive""",50,45,70,144000,100,90
6,"""C""",28,52000,"""active""",60,55,56,104000,120,110


## `sort()`

In [10]:
df1.sort("income", descending=True) 

id,group,age,income,status,prefix_score1,prefix_score2
i64,str,i64,i64,str,i64,i64
4,"""B""",40,80000,"""active""",40,35
5,"""C""",35,72000,"""inactive""",50,45
2,"""A""",30,60000,"""inactive""",20,15
6,"""C""",28,52000,"""active""",60,55
1,"""A""",25,50000,"""active""",10,5
3,"""B""",22,45000,"""active""",30,25


## `.rename()`

In [11]:
df1.rename({"income": "annual_income"}).select(['id', 'age', 'group', 'annual_income'])

id,age,group,annual_income
i64,i64,str,i64
1,25,"""A""",50000
2,30,"""A""",60000
3,22,"""B""",45000
4,40,"""B""",80000
5,35,"""C""",72000
6,28,"""C""",52000


## `.slice()`


In [12]:
df1.slice(0, 3)

id,group,age,income,status,prefix_score1,prefix_score2
i64,str,i64,i64,str,i64,i64
1,"""A""",25,50000,"""active""",10,5
2,"""A""",30,60000,"""inactive""",20,15
3,"""B""",22,45000,"""active""",30,25


## `.select()` with aggregation

In [13]:
df1.select(
    pl.col("income").mean().round().alias("avg_income"),
    pl.col("age").max().alias("max_age"),
)

avg_income,max_age
f64,i64
59833.0,40


## `.group_by()`

In [14]:
df1.group_by("status").agg(
    pl.col("income").mean().round(decimals=2).alias("Average Income"),
    pl.col('age').min().alias("Young_Gens")
)

status,Average Income,Young_Gens
str,f64,i64
"""active""",56750.0,22
"""inactive""",66000.0,30


## `.group_by().count()`

In [15]:
df1.group_by('status').len()

status,len
str,u32
"""active""",4
"""inactive""",2


## Left join `how = 'left'`

In [16]:
df_combined_left = df1.join(df2, on = "id", how="left")
df_combined_left

id,group,age,income,status,prefix_score1,prefix_score2,city
i64,str,i64,i64,str,i64,i64,str
1,"""A""",25,50000,"""active""",10,5,"""NY"""
2,"""A""",30,60000,"""inactive""",20,15,"""LA"""
3,"""B""",22,45000,"""active""",30,25,"""Dallas"""
4,"""B""",40,80000,"""active""",40,35,null
5,"""C""",35,72000,"""inactive""",50,45,null
6,"""C""",28,52000,"""active""",60,55,null


## Right join `how = 'right'`

In [17]:
df_combined_right = df1.join(df2, on="id", how="right")
df_combined_right

group,age,income,status,prefix_score1,prefix_score2,id,city
str,i64,i64,str,i64,i64,i64,str
"""A""",25,50000,"""active""",10,5,1,"""NY"""
"""A""",30,60000,"""inactive""",20,15,2,"""LA"""
"""B""",22,45000,"""active""",30,25,3,"""Dallas"""
null,null,null,null,null,null,7,"""Houston"""


## Inner join `how = "inner"`

In [18]:
df_combined_inner = df1.join(df2, on="id", how="inner")
df_combined_inner

id,group,age,income,status,prefix_score1,prefix_score2,city
i64,str,i64,i64,str,i64,i64,str
1,"""A""",25,50000,"""active""",10,5,"""NY"""
2,"""A""",30,60000,"""inactive""",20,15,"""LA"""
3,"""B""",22,45000,"""active""",30,25,"""Dallas"""


## Outer join `how = "inner"`

In [19]:
df_combined_inner = df1.join(df2, on="id", how="inner")
df_combined_inner

id,group,age,income,status,prefix_score1,prefix_score2,city
i64,str,i64,i64,str,i64,i64,str
1,"""A""",25,50000,"""active""",10,5,"""NY"""
2,"""A""",30,60000,"""inactive""",20,15,"""LA"""
3,"""B""",22,45000,"""active""",30,25,"""Dallas"""


## Selectors
### `pl.all()`, `pl.col()`

- select by data type


In [20]:
df_combined_left.select(
    pl.col(pl.Int64)
)

id,age,income,prefix_score1,prefix_score2
i64,i64,i64,i64,i64
1,25,50000,10,5
2,30,60000,20,15
3,22,45000,30,25
4,40,80000,40,35
5,35,72000,50,45
6,28,52000,60,55


In [21]:
df_combined_left.select(pl.all().exclude(["income_k"]))

id,group,age,income,status,prefix_score1,prefix_score2,city
i64,str,i64,i64,str,i64,i64,str
1,"""A""",25,50000,"""active""",10,5,"""NY"""
2,"""A""",30,60000,"""inactive""",20,15,"""LA"""
3,"""B""",22,45000,"""active""",30,25,"""Dallas"""
4,"""B""",40,80000,"""active""",40,35,null
5,"""C""",35,72000,"""inactive""",50,45,null
6,"""C""",28,52000,"""active""",60,55,null


## Chained conditional

In [22]:
df_combined_left.with_columns(
    pl.when(pl.col("age") < 30).then(pl.lit("young"))
    .when(pl.col('age') < 40)
    .then(pl.lit("Mid"))
    .otherwise(pl.lit("Senior"))
    .alias("age_group")
)

id,group,age,income,status,prefix_score1,prefix_score2,city,age_group
i64,str,i64,i64,str,i64,i64,str,str
1,"""A""",25,50000,"""active""",10,5,"""NY""","""young"""
2,"""A""",30,60000,"""inactive""",20,15,"""LA""","""Mid"""
3,"""B""",22,45000,"""active""",30,25,"""Dallas""","""young"""
4,"""B""",40,80000,"""active""",40,35,null,"""Senior"""
5,"""C""",35,72000,"""inactive""",50,45,null,"""Mid"""
6,"""C""",28,52000,"""active""",60,55,null,"""young"""


## Prefix selection

In [23]:
df_combined_inner.select(pl.col("^prefix.*$"))

prefix_score1,prefix_score2
i64,i64
10,5
20,15
30,25


## Regex in `pl.col()`
- selecting the all columns containing _score_ in the dataframe

In [24]:
df_combined_inner.select(pl.col("^.*score.*$"))

prefix_score1,prefix_score2
i64,i64
10,5
20,15
30,25


__using polars selector__

In [25]:
import polars.selectors as cs

df_combined_inner.select(cs.matches("score"))

prefix_score1,prefix_score2
i64,i64
10,5
20,15
30,25
